In [ ]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from keras.constraints import maxnorm

from sklearn import pipeline
from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
X_treinamento = pd.read_json(r'X_treinamento.json')
X_teste = pd.read_json      (r'X_teste.json')
treino_json = pd.read_json  (r'y_treinamento.json', typ='series')
teste_json = pd.read_json   (r'y_teste.json', typ='series')

In [ ]:
a = []
b = []

for i in range(treino_json.size):
    a.append(list(treino_json[i]))
    
for i in range(teste_json.size):
    b.append(list(teste_json[i]))

In [ ]:
y_treinamento = np.array(a, dtype='float32')
y_teste = np.array(b, dtype='float32')

In [ ]:
def build_model(
    optimizer,
    learning_rate,
    activation,
    init_mode,
    dropout_rate,
    weight_constraint,
    neurons,
    regularizers
):
    model = models.Sequential()
    
    # layer 1
    model.add(
        layers.Dense(
            neurons,
            activation = activation,
            kernel_initializer=init_mode,
            kernel_constraint=maxnorm(weight_constraint),
            kernel_regularizer=regularizers,
            input_shape=[len(X_treinamento.keys())]
        )
    )
    model.add(layers.Dropout(dropout_rate))
    
    # layer 2
    model.add(layers.Dense(256))
    
    model.compile(
        loss='mse',
        optimizer=optimizer(learning_rate),
        metrics=['mae', 'mse', keras.metrics.RootMeanSquaredError()]
    )
    
    return model

In [ ]:
d = {}
with open(r'best-params.txt') as f:
    for line in f:
       (key, val) = line.split(", ")
       d[key] = val.strip()

In [ ]:
scaled_X_treinamento = preprocessing.PowerTransformer(method='yeo-johnson').fit_transform(X_treinamento)
scaled_y_treinamento = preprocessing.PowerTransformer(method='yeo-johnson').fit_transform(y_treinamento)

In [ ]:
scaled_X_teste = preprocessing.PowerTransformer(method='yeo-johnson').fit_transform(X_teste)
scaled_y_teste = preprocessing.PowerTransformer(method='yeo-johnson').fit_transform(y_teste)

In [ ]:
model = build_model(
    tf.keras.optimizers.Adagrad,
    float(d['learning_rate']),
    d['activation'],
    d['init_mode'],
    float(d['dropout_rate']),
    int(d['weight_constraint']),
    int(d['neurons']),
    tf.keras.regularizers.l2(0.0001)
)

In [ ]:
history = model.fit(
    scaled_X_treinamento,
    scaled_y_treinamento,
    epochs=int(d['epochs']),
    batch_size=int(d['batch_size']),
    validation_data=(scaled_X_teste, scaled_y_teste), 
    verbose=1
)

In [ ]:
sns.set_theme()

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Épocas')
    plt.ylabel('MAE')
    plt.plot(hist['epoch'], hist['mae'], label='Erro Treinamento')
    plt.plot(hist['epoch'], hist['val_mae'], label = 'Erro Validação')
    plt.ylim([0,1])
    plt.legend()

    plt.figure()
    plt.xlabel('Épocas')
    plt.ylabel('MSE')
    plt.plot(hist['epoch'], hist['mse'], label='Erro Treinamento')
    plt.plot(hist['epoch'], hist['val_mse'], label = 'Erro Validação')
    plt.ylim([0,1])
    plt.legend()
    plt.show()

plot_history(history)